In [28]:
import json
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import imshow
import numpy as np
import cv2
%matplotlib inline
input_dir_path='mpii_human_pose_v1\\input_images\\'
output_dir_path='mpii_human_pose_v1\\output_images360x480\\'
target_img_height=480
target_img_width=360

In [29]:
raw_datas= open('train_data\\data.json').readlines()
json_fp = open('train_data\\new_data.json', 'w')

In [30]:
loop_times=0
for raw_data in raw_datas:
    if loop_times%100==0:
        print(loop_times)
    data=json.loads(raw_data.strip())      #讀標記資料第?筆
    img = cv2.imread(input_dir_path+data['filename']) #讀圖檔
    joints_pos=data['joint_pos']               #讀關節點資料
    original_img_height, original_img_width, channels = img.shape        #獲得圖片資訊
    original_data_scale=data['scale']
    original_data_objpos=data['objpos']
    
                                                                   #算bounding box的位置
    side=original_data_scale*200*1.1 
    left_top_X=int(original_data_objpos[0]-(side/2))
    left_top_Y=int(original_data_objpos[1]-(side/2))
    right_down_X=min(original_img_width,left_top_X+int(side))
    right_down_Y=min(original_img_height,left_top_Y+int(side))
    left_top_X=max(0,left_top_X)
    left_top_Y=max(0,left_top_Y)

    new_side_width=right_down_X-left_top_X
    new_side_height=right_down_Y-left_top_Y

    crop_img=img[left_top_Y:right_down_Y,left_top_X:right_down_X]   #切割bound box的範圍
    for joint in joints_pos:
        joints_pos[str(joint)][0]=int(joints_pos[str(joint)][0]-left_top_X)
        joints_pos[str(joint)][1]=int(joints_pos[str(joint)][1]-left_top_Y)
                                                                        #first calculate padding
    left_padding=int(target_img_width/2-new_side_width/2)
    top_padding=int(target_img_height/2-new_side_height/2)
    right_padding=int(target_img_width-left_padding-new_side_width)
    bottom_padding=int(target_img_height-top_padding-new_side_height)

    if left_padding<0 or top_padding<0 or right_padding<0 or bottom_padding<0: #crop_image too big
        ratio_resize_X=float(target_img_width)/float(new_side_width)
        ratio_resize_Y=float(target_img_height)/float(new_side_height)
        ratio_resize=min(ratio_resize_X,ratio_resize_Y)
        crop_img=cv2.resize(crop_img,(0,0),fx=ratio_resize,fy=ratio_resize)
        crop_img_height, crop_img_width, channels = crop_img.shape   #獲得crop圖片資訊
        left_padding=int(target_img_width/2-crop_img_width/2)
        top_padding=int(target_img_height/2-crop_img_height/2)
        right_padding=int(target_img_width-left_padding-crop_img_width)
        bottom_padding=int(target_img_height-top_padding-crop_img_height)
        for joint in joints_pos:
            joints_pos[str(joint)][0]=(joints_pos[str(joint)][0]*ratio_resize)+left_padding
            joints_pos[str(joint)][1]=(joints_pos[str(joint)][1]*ratio_resize)+top_padding
    else:
        for joint in joints_pos:
            joints_pos[str(joint)][0]=int(joints_pos[str(joint)][0]+left_padding)
            joints_pos[str(joint)][1]=int(joints_pos[str(joint)][1]+top_padding)

    padding_crop_img= cv2.copyMakeBorder(
        crop_img,top=top_padding,bottom=bottom_padding
        ,left=left_padding,right=right_padding
        ,borderType=cv2.BORDER_CONSTANT,value=[0,0,0])
    
    cv2.imwrite(output_dir_path+str(loop_times)+"-"+data['filename'],padding_crop_img)

    data_output = {
        "filename": str(loop_times)+"-"+data['filename'],
        "train": data['train'],
        "is_visible": data['is_visible'],
        "joint_pos": joints_pos
    }
    print(json.dumps(data_output), file=json_fp)
    loop_times=loop_times+1
print("end")
json_fp.close()

0
end
